In [ ]:
from pymilvus import MilvusClient

# Initialize Milvus Lite (local file)
atlas = MilvusClient("geospatial_atlas.db")

# Create Collection
if not atlas.has_collection("world_anchors"):
    atlas.create_collection(
        collection_name="world_anchors",
        dimension=64, #alphaearth 64d
        metric_type="COSINE",
        auto_id=True
    )

def index_anchors(batch_alpha_vectors, batch_metadata):
    data = [
        {"vector": vec, "lat": meta['lat'], "lon": meta['lon'], "s2": meta['s2']}
        for vec, meta in zip(batch_alpha_vectors, batch_metadata)
    ]
    atlas.insert(collection_name="world_anchors", data=data)

In [ ]:
def geolocate(image_path, top_k=5):
    # 1. Image -> StreetCLIP -> 768D Vector
    visual_vec = get_streetclip_embedding(image_path) 
    
    scout.eval()
    with torch.no_grad():
        query_vector = scout(torch.tensor(visual_vec).to(device)).cpu().numpy()

    # 3. 64D Vector -> Milvus Atlas Search
    results = atlas.search(
        collection_name="world_anchors",
        data=[query_vector],
        limit=top_k,
        output_fields=["lat", "lon", "s2"]
    )
    
    return results[0]

figure out how we should interact with it.